In [48]:
import psycopg2
import pandas as pd
import pickle

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

pd.set_option('display.max_columns', None)

#### Query data from PostgreSQL table, save to csv

In [105]:
conn = psycopg2.connect(host="localhost", dbname='cs', user='postgres', password='123456', port=5432)
cur = conn.cursor()

query = """SELECT * FROM side_dataset where id>=684"""
cur.execute(query)

rows = cur.fetchall()
columns = [desc[0] for desc in cur.description]
df = pd.DataFrame(rows, columns=columns)

cur.close()
conn.close()

df.to_csv('df_642.csv')

#### Begin

In [41]:
data = pd.read_csv('df_642.csv')
#First 4 rows are not needed
df = data.iloc[:,5:]

#### While parsing the data, I was storing this particular stat (average rounds lost(won) on a map) as 0's in case of missing data. So, I convert them to NULL, to deal with them further. Also deelting the rows with incorrect data

In [42]:
df.loc[(df['t1_rounds_lost']>0) & (df['t1_rounds_won'] == 0), 't1_rounds_won'] = None
df.loc[(df['t1_rounds_lost']==0) & (df['t1_rounds_won'] > 0), 't1_rounds_lost'] = None
df.loc[(df['t2_rounds_lost']==0) & (df['t2_rounds_won'] > 0), 't2_rounds_lost'] = None
df.loc[(df['t2_rounds_lost']>0) & (df['t2_rounds_won'] == 0), 't2_rounds_won'] = None

df.drop(df[(df['t2_5v4']==0) & (df['t2_played']>0)].index, axis=0, inplace=True)
df.drop(df[(df['t1_5v4']==0) & (df['t1_played']>0)].index, axis=0, inplace=True)

#### Try filling NULL values with 13's

In [43]:
df['t1_rounds_lost'].fillna(13, inplace=True)
df['t1_rounds_won'].fillna(13, inplace=True)
df['t2_rounds_lost'].fillna(13, inplace=True)
df['t2_rounds_won'].fillna(13, inplace=True)

#### The differences between team stats will be the predicting features, so creating the features

In [44]:
cols_name = [df.columns[i]+'_dif' for i in range(0,len(df.columns[:22]),2)] + [df.columns[i]+'_dif' for i in range(22, 24)] + [df.columns[i]+'_dif' for i in range(25, 33)]

dataset = pd.DataFrame(columns=cols_name)

dataset = pd.DataFrame(columns=cols_name)
for index, name in enumerate(cols_name[:11]):
    dataset[name] = df[df.columns[index*2]] - df[df.columns[index*2+1]]

for index, name in enumerate(cols_name[11:13]):
    dataset[name] = df[df.columns[22+index]] - df[df.columns[33+index]]

for index, name in enumerate(cols_name[13:]):
    dataset[name] = df[df.columns[25+index]] - df[df.columns[36+index]]

dataset['t1_fp'] = df['t1_fp']
dataset['t2_fp'] = df['t2_fp']
dataset['result'] = df['result']

#### Preparing data

In [46]:
scaler = StandardScaler()

columns = ['t1_winstreak_dif','t1_h2h_dif','t1_ranking_dif','t1_pluses_dif','t1_minuses_dif','t1_coef_dif','t1_rating_dif','t1_event_rating_dif', \
            't1_num_maps_dif','t1_avg_lost_dif','t1_avg_won_dif','t1_rounds_lost_dif','t1_rounds_won_dif','t1_fp_percent_dif','t1_winrate_dif', \
            't1_played_dif','t1_map_winstreak_dif','t1_map_losestreak_dif','t1_5v4_dif','t1_4v5_dif','t1_pistol_dif']

scaler.fit(dataset.loc[:,columns])
dataset.loc[:,columns] = scaler.transform(dataset.loc[:,columns])

with open('scaler.pkl','wb') as f:
    pickle.dump(scaler, f)

In [58]:
Y = dataset.loc[:,['result']]
X = dataset.drop('result',axis=1)

x_train, x_rest, y_train, y_rest = train_test_split(X, Y, test_size=200, shuffle=True, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_rest, y_rest, test_size=100, shuffle=True, random_state=42)
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(419, 23)
(100, 23)
(100, 23)
